In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import feather
import datetime as dt
from tqdm import tqdm_notebook

In [2]:
PATH = '../src/'
!ls {PATH}

TOTH_ACCOUNT_PROFILE.7z  TOTH_PAYMENT_PRODUCT.7z  df_accs_samp-raw
TOTH_ACCOUNT_PROFILE.txt TOTH_PRODUCT.7z          df_invs_samp-raw
TOTH_INVOICE.7z          TOTH_PRODUCT.txt         df_pays_samp-raw
TOTH_INVOICE.txt         TOTH_SCOMS.7z            df_prods_samp-raw
TOTH_PAYMENT.txt         TOTH_SCOMS.txt           df_scoms_samp-raw


In [3]:
# import all the data
df_accs = feather.read_dataframe(f'{PATH}df_accs_samp-raw')
df_invs = feather.read_dataframe(f'{PATH}df_invs_samp-raw')
df_prods = feather.read_dataframe(f'{PATH}df_prods_samp-raw')
df_scoms = feather.read_dataframe(f'{PATH}df_scoms_samp-raw')
df_pays = feather.read_dataframe(f'{PATH}df_accs_samp-raw')

In [4]:
df_scoms.head()

,TEL,FAILDATE,COMDATE,CHKWORK_0,CHKWORK_AN,CHKWORK_BN,CHKWORK_CN,CHKWORK_FN,CHKWORK_HN,CHKWORK_IN,...,FAULTRANGE_มากกว่า 1 - 2 ชม.,FAULTRANGE_มากกว่า 1 วัน - 7 วัน,FAULTRANGE_มากกว่า 2 - 3 ชม.,FAULTRANGE_มากกว่า 3 - 3.5 ชม.,FAULTRANGE_มากกว่า 3.5 - 4 ชม.,FAULTRANGE_มากกว่า 4 - 6 ชม.,FAULTRANGE_มากกว่า 6 - 24 ชม.,FAULTRANGE_มากกว่า 7 วัน,FAULTRANGE_อยู่ระหว่างดำเนินการ,FAULTRANGE_nan
0,9DAEFFFCABF29AF26195EA22D6BFC904,2017-02-13 18:49:23,2017-02-14 10:50:07,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,E0254B612E7532531F6CC4D053A71C5B,2017-02-13 18:50:00,2017-02-14 11:19:05,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,99F142C6C271C44E0CC78E9A31766AB1,2017-02-13 18:51:06,2017-02-13 19:29:02,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14E0F93F5C1C19814B328DFE353B0D3E,2017-02-13 18:51:25,2017-02-16 08:47:09,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,CFC4B4F105E0BC4100C16B3BB8042F80,2017-02-13 18:51:29,2017-02-14 10:14:39,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [7]:
# create rolling statistic (monthly) for scoms data
df_scoms['CMPTIME'] = df_scoms.COMDATE - df_scoms.FAILDATE

In [8]:
# check the date range
print(df_scoms.FAILDATE.min(),' - ',df_scoms.FAILDATE.max())

2017-01-01 00:09:05  -  2018-12-31 23:46:49


In [12]:
for year, month in tqdm_notebook(zip([2017]*12 + [2018]*12, list(range(1,13))*2)):
    df_scoms_temp = df_scoms.copy()
    df_scoms_temp['elapse'] = dt.datetime(year,month,1) - df_scoms_temp.FAILDATE
    df_scoms_temp['elapse'] = df_scoms_temp.elapse.dt.days
    df_scoms_temp = df_scoms_temp.loc[df_scoms_temp.elapse>0]
    df_scoms_temp.to_csv('../bin/monthly/df_scoms_'+str(year).zfill(4)+str(month).zfill(2)+'.csv',index=False)

In [13]:
df_prods.head()

,ACCOUNT,TEL,TEL_START_DT,TEL_END_DT,TARIFF,QUANTITY,CHARGE
0,B2A9B9930AAB063D7B6F88FBBCC4C40D,50DC72E95344D07C0C095422F482A067,2018-09-06,2021-09-05,Plan 5354,1,450.0
1,B2A9B9930AAB063D7B6F88FBBCC4C40D,EA0AC7D21E5067E42BAACA733BB59962,2016-07-24,2016-07-26,Plan 3742,1,590.0
2,B2A9B9930AAB063D7B6F88FBBCC4C40D,EA0AC7D21E5067E42BAACA733BB59962,2016-07-27,2017-10-11,Plan 618,1,590.0
3,B2A9B9930AAB063D7B6F88FBBCC4C40D,EA0AC7D21E5067E42BAACA733BB59962,2017-10-12,2018-10-21,Plan 513,1,650.0
4,BDC469E8E9F835C005922DE8333905E7,AB4F21ECADFB6AF25AA991A8101130F2,2013-06-06,2018-08-10,Plan 5180,1,390.0


In [14]:
# create rolling statistic (monthly) for prods data
for year, month in tqdm_notebook(zip([2017]*12 + [2018]*12, list(range(1,13))*2)):
    df_prods_temp = df_prods.copy()
    df_prods_temp['elapse_start'] =  dt.datetime(year,month,1) - df_prods_temp.TEL_START_DT
    df_prods_temp['elapse_start'] = df_prods_temp.elapse_start.dt.days
    df_prods_temp = df_prods_temp.loc[df_prods_temp.elapse_start>0]
    df_prods_temp['elapse_end'] = dt.datetime(year,month,1) - df_prods_temp.TEL_END_DT
    df_prods_temp['elapse_end'] = df_prods_temp.elapse_end.dt.days
    df_prods_temp['elapse_end'] = df_prods_temp.elapse_end.clip(-1, np.inf).fillna(-1)
    df_prods_temp.to_csv('../bin/monthly/df_product_'+str(year).zfill(4)+str(month).zfill(2)+'.csv',index=False)

In [16]:
del df_scoms, df_prods

In [17]:
df_accs.head()

,ACCOUNT,START_DT,BILL_DELIVER,PAY_METHOD,PROVINCE
0,BDC469E8E9F835C005922DE8333905E7,2011-10-21,Paper Bill,Cash,จังหวัดสุราษฎร์ธานี
1,11905624254857E8415B2323649CA0B5,2014-02-18,Paper Bill,Cash,จังหวัดตรัง
2,D241D5652995AE1ABDB246B39ECDF150,2017-08-16,No Bill,Cash,จังหวัดกาฬสินธุ์
3,799E80FDF19DECFD409583545A12B157,2017-07-06,No Bill,Cash,จังหวัดสุราษฎร์ธานี
4,2B4132D1BD70AEF027122F1A91F75A81,2018-02-08,Paper Bill,Cash,จังหวัดหนองบัวลำภู


In [18]:
# preprocess account data
df_accs.BILL_DELIVER.value_counts()

Paper Bill    221585
No Bill        36163
E-Bill          8964
SMS P-Bill        11
SMS E-Bill         2
Name: BILL_DELIVER, dtype: int64

In [19]:
df_accs.PAY_METHOD.value_counts()

Cash                   244396
EFT (Direct Debit)      17338
CPM (Direct Credit)      4991
Name: PAY_METHOD, dtype: int64

In [20]:
df_accs.PROVINCE.nunique()

265

In [21]:
df_accs = df_accs.drop('START_DT',axis=1)
df_accs.to_feather('df_accs_samp-raw')

In [4]:
# create rolling statistic (monthly) for invoice data
df_invs.head()

,ACCOUNT,INVOICE,BILL_DT,DUE_DATE,SETTLED_DT,AMT,VAT,TOTAL,ADJUST,PAYMENT,OUTSTANDING
0,B2A9B9930AAB063D7B6F88FBBCC4C40D,E645402650BF5ABF19140B45873986DB,JUN-14,7/7/2014,7/31/2014,1763.50,123.45,1886.95,NaN,1886.95,0.0
1,B2A9B9930AAB063D7B6F88FBBCC4C40D,1A3BA86FCA850BA121845F21D8683A35,AUG-14,9/7/2014,9/25/2014,1556.83,108.98,1665.81,NaN,1665.81,0.0
2,B2A9B9930AAB063D7B6F88FBBCC4C40D,DFA66608782FCB4CD887843B0755C7B9,SEP-14,10/7/2014,9/25/2014,1866.00,130.62,1996.62,NaN,1996.62,0.0
3,B2A9B9930AAB063D7B6F88FBBCC4C40D,25F3855660E98F52855318DA06761EAA,OCT-14,11/7/2014,11/3/2014,1650.00,115.50,1765.50,NaN,1765.50,0.0
4,B2A9B9930AAB063D7B6F88FBBCC4C40D,6D27F1EF6E5CFC14DF69BBA6AB305131,DEC-14,1/7/2015,1/6/2015,1850.00,129.50,1979.50,NaN,1979.50,0.0


In [22]:
df_invs['BILL_DT'] = pd.to_datetime(df_invs.BILL_DT,format='%b-%y')
df_invs['DUE_DATE'] = pd.to_datetime(df_invs.DUE_DATE,format='%m/%d/%Y')
df_invs['SETTLED_DT'] = pd.to_datetime(df_invs.SETTLED_DT,format='%m/%d/%Y')
df_invs['late_payment_days'] = df_invs['SETTLED_DT'] - df_invs['DUE_DATE']
df_invs['late_payment_days'] = df_invs.late_payment_days.dt.days

In [23]:
for year, month in tqdm_notebook(zip([2017]*12 + [2018]*12, list(range(1,13))*2)):
    df_invs_temp = df_invs.copy()
    df_invs_temp['elapse_settled'] = dt.datetime(year,month,1) - df_invs.SETTLED_DT
    df_invs_temp['elapse_settled'] = df_invs_temp.elapse_settled.dt.days
    df_invs_temp = df_invs_temp.loc[df_invs_temp.elapse_settled>0]
    df_invs_temp['elapse_due'] = dt.datetime(year,month,1) - df_invs_temp.DUE_DATE
    df_invs_temp['elapse_due'] = df_invs_temp.elapse_due.dt.days
    df_invs_temp.to_csv('../bin/monthly/df_invs_'+str(year).zfill(4)+str(month).zfill(2)+'.csv',index=False)

In [7]:
df_invs.head(150)

,ACCOUNT,INVOICE,BILL_DT,DUE_DATE,SETTLED_DT,AMT,VAT,TOTAL,ADJUST,PAYMENT,OUTSTANDING
0,B2A9B9930AAB063D7B6F88FBBCC4C40D,E645402650BF5ABF19140B45873986DB,JUN-14,7/7/2014,7/31/2014,1763.50,123.45,1886.95,NaN,1886.95,0.0
1,B2A9B9930AAB063D7B6F88FBBCC4C40D,1A3BA86FCA850BA121845F21D8683A35,AUG-14,9/7/2014,9/25/2014,1556.83,108.98,1665.81,NaN,1665.81,0.0
2,B2A9B9930AAB063D7B6F88FBBCC4C40D,DFA66608782FCB4CD887843B0755C7B9,SEP-14,10/7/2014,9/25/2014,1866.00,130.62,1996.62,NaN,1996.62,0.0
3,B2A9B9930AAB063D7B6F88FBBCC4C40D,25F3855660E98F52855318DA06761EAA,OCT-14,11/7/2014,11/3/2014,1650.00,115.50,1765.50,NaN,1765.50,0.0
4,B2A9B9930AAB063D7B6F88FBBCC4C40D,6D27F1EF6E5CFC14DF69BBA6AB305131,DEC-14,1/7/2015,1/6/2015,1850.00,129.50,1979.50,NaN,1979.50,0.0
5,B2A9B9930AAB063D7B6F88FBBCC4C40D,732F660AF93683CF9F691CF8E84F70FC,JAN-15,2/7/2015,3/10/2015,1850.00,129.50,1979.50,NaN,1979.50,0.0
6,B2A9B9930AAB063D7B6F88FBBCC4C40D,7592E5968907DBEADFF826A081B4E2EB,FEB-15,3/7/2015,3/10/2015,1856.00,129.92,1985.92,NaN,1985.92,0.0
7,B2A9B9930AAB063D7B6F88FBBCC4C40D,1278620885FBC93EA048544DC76F1A98,MAR-15,4/7/2015,5/14/2015,1850.00,129.50,1979.50,NaN,1979.50,0.0
8,B2A9B9930AAB063D7B6F88FBBCC4C40D,B225572D1E37AE77D698961D0EDAB879,MAY-15,6/7/2015,7/14/2015,1650.00,115.50,1765.50,NaN,1765.50,0.0
9,B2A9B9930AAB063D7B6F88FBBCC4C40D,2ED1AE3B6521FAB14AB77A63865000C0,JUL-15,8/7/2015,9/14/2015,1650.00,115.50,1765.50,NaN,1765.50,0.0
